<a href="https://colab.research.google.com/github/krishnamishra2610/Deep-Learning/blob/main/Optimum_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 97 kB 4.6 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_minist = keras.datasets.fashion_mnist

In [4]:
(train_img,train_label),(test_img,test_label) = fashion_minist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
train_img.shape

(60000, 28, 28)

##Scaling 

In [6]:
train_img = train_img/255.0
test_img = test_img/255.0

In [7]:
train_img = train_img.reshape(len(train_img),28,28,1)
test_img = test_img.reshape(len(test_img),28,28,1)

In [8]:
output_label = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

### **Using keras_tuner for hyper parameter optimization**

In [13]:
def build(hp):
  model = keras.Sequential([
                            #1st covolutional layer (input layer)
                            keras.layers.Conv2D(
                                filters =hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                                kernel_size = hp.Choice('conv_1_kernal',values=[3,4,5]),
                                activation='relu',
                                input_shape=(28,28,1)
                            ),
                            keras.layers.MaxPool2D(pool_size=(2,2)),

                            #2nd convolution layer(hidden layer)
                             keras.layers.Conv2D(
                                filters =hp.Int('conv_2_filter',min_value=32,max_value=128,step=16),
                                kernel_size = hp.Choice('conv_2_kernal',values=[3,4,5]),
                                activation='relu',

                            ),
                            keras.layers.MaxPool2D(pool_size=(2,2)),

                            #fully connected classifier

                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units = hp.Int('dense_1_units',min_value=32,max_value=2048,step=16),
                                activation ='relu'
                            ),
                            #output layer
                            keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics =['accuracy'])
  return model

In [14]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [23]:
tuner_search =  RandomSearch(build,objective='accuracy',max_trials=3,directory='output1',project_name='Minist_fashion')

In [24]:
tuner_search.search(train_img,train_label,epochs=2)

Trial 3 Complete [00h 00m 41s]
accuracy: 0.8576499819755554

Best accuracy So Far: 0.8998500108718872
Total elapsed time: 00h 01m 55s
INFO:tensorflow:Oracle triggered exit


In [25]:
model = tuner_search.get_best_models(1)[0]

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 112)       64624     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 112)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2800)              0         
_________________________________________________________________
dense (Dense)                (None, 1856)              5198656   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

In [27]:
model.fit(train_img,train_label,epochs=10,validation_split=0.1,initial_epoch=2)

Epoch 3/10
1688/1688 [==============================] - 15s 8ms/step - loss: 0.2188 - accuracy: 0.9172 - val_loss: 0.2126 - val_accuracy: 0.9245
Epoch 4/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1814 - accuracy: 0.9326 - val_loss: 0.2255 - val_accuracy: 0.9143
Epoch 5/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1517 - accuracy: 0.9436 - val_loss: 0.2167 - val_accuracy: 0.9203
Epoch 6/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1237 - accuracy: 0.9530 - val_loss: 0.2413 - val_accuracy: 0.9185
Epoch 7/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1045 - accuracy: 0.9606 - val_loss: 0.2708 - val_accuracy: 0.9155
Epoch 8/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0826 - accuracy: 0.9690 - val_loss: 0.2659 - val_accuracy: 0.9182
Epoch 9/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0718 - accuracy: 0.9729 - val_loss: 0.2870 - val_ac

In [28]:
model.predict(test_img[[0]])

array([[7.44938428e-18, 2.81897131e-17, 8.64162630e-20, 3.89413926e-18,
        5.24511670e-18, 4.04309053e-14, 1.06737275e-16, 1.23318824e-14,
        5.22615225e-22, 1.00000000e+00]], dtype=float32)

In [29]:
np.argmax(model.predict(test_img[[0]]))

9

## Predicted class

In [33]:
output_label[np.argmax(model.predict(test_img[[100]]))]

'Dress'

## Actual class

In [34]:
output_label[test_label[100]]

'Dress'